# Klavis Google Cloud MCP Sandbox API Example

This notebook demonstrates the core sandbox lifecycle endpoints for **Google Cloud Toolathlon** (within local sandbox):
- **Acquire** a local sandbox with the `google_cloud` MCP server
- **Get** sandbox details *(optional — acquire already returns everything)*
- **Call MCP tools** for BigQuery, Cloud Storage, Cloud Logging, and Compute Engine using the **Python MCP SDK**
- **Release** the sandbox

## 1. Setup

In [ ]:
import os
import json
import httpx
from dotenv import load_dotenv

load_dotenv()

BASE_URL = "https://api.klavis.ai"
KLAVIS_API_KEY = os.environ.get("KLAVIS_API_KEY")

headers = {"Authorization": f"Bearer {KLAVIS_API_KEY}"}

SERVER_NAME = "google_cloud"

## 2. Acquire a Local Sandbox with Google Cloud

`POST /local-sandbox` — Acquire a local sandbox VM with the `google_cloud` MCP server.

The local sandbox API spins up a VM hosting local MCP servers that need direct access to GCP services.

In [ ]:
async with httpx.AsyncClient(timeout=120.0) as client:
    resp = await client.post(
        f"{BASE_URL}/local-sandbox",
        headers=headers,
        json={"server_names": ["google_cloud"], "benchmark": "Toolathlon"}
    )

acquire_data = resp.json()
LOCAL_SANDBOX_ID = acquire_data["local_sandbox_id"]

# Extract the Google Cloud MCP server info from the acquire response
gc_server = next(
    s for s in acquire_data["servers"]
    if s["server_name"] == "google_cloud"
)

MCP_SERVER_URL = gc_server["mcp_server_url"]
AUTH_DATA = gc_server.get("auth_data", {})

print(f"Local Sandbox ID: {LOCAL_SANDBOX_ID}")
print(f"MCP Server URL: {MCP_SERVER_URL}")
print(f"Auth data: {json.dumps(AUTH_DATA, indent=2)}")

## 3. Get Sandbox Details *(Optional)*

> The acquire response already returns all server details. These endpoints are useful if you need to re-fetch info later.

`GET /local-sandbox/{local_sandbox_id}` — Get details for a specific local sandbox

In [ ]:
# Get details for our sandbox
async with httpx.AsyncClient() as client:
    resp = await client.get(
        f"{BASE_URL}/local-sandbox/{LOCAL_SANDBOX_ID}",
        headers=headers
    )
sandbox_info = resp.json()
print(json.dumps(sandbox_info, indent=2))

## 4. Call MCP Tools via the Python MCP SDK

The Google Cloud MCP server runs inside the local sandbox and speaks the MCP Streamable HTTP protocol.

Unlike the email MCP server, **no custom headers** are needed when connecting — authentication is handled internally by the sandbox via service account credentials.

### 4a. List Available Tools

List all MCP tools exposed by the Google Cloud MCP server.

In [ ]:
from mcp.client.streamable_http import streamable_http_client
from mcp import ClientSession

print(f"MCP Server URL: {MCP_SERVER_URL}")

async with streamable_http_client(MCP_SERVER_URL) as (read, write, _):
    async with ClientSession(read, write) as session:
        await session.initialize()
        tools = await session.list_tools()
        print(f"Available tools ({len(tools.tools)}):")
        for tool in tools.tools:
            print(f"  - {tool.name}: {tool.description[:80]}...")

### 4b. Compute Engine — List Zones

List all available Compute Engine zones.

In [ ]:
async with streamable_http_client(MCP_SERVER_URL) as (read, write, _):
    async with ClientSession(read, write) as session:
        await session.initialize()
        result = await session.call_tool("compute_list_zones", {})
        for content in result.content:
            print(content.text)

## 5. Release Local Sandbox

`DELETE /local-sandbox/{local_sandbox_id}` — Release the local sandbox VM and all associated servers

In [ ]:
async with httpx.AsyncClient() as client:
    resp = await client.delete(
        f"{BASE_URL}/local-sandbox/{LOCAL_SANDBOX_ID}",
        headers=headers
    )

print(resp.json())